In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from joblib import dump,load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
datapath = './verify/'
binary_clf_path = './classifiers/binary/'
multi_clf_path = './classifiers/multiclass/'

In [3]:
#for binary classification
bpca = load(f'{binary_clf_path}bpca.joblib')
bscaler = load(f'{binary_clf_path}bscaler.joblib')
bclf = load(f'{binary_clf_path}binary.joblib')

In [4]:
#for multi binary classfication
dos = load(f'{multi_clf_path}dos')
ddos = load(f'{multi_clf_path}ddos')
portscan = load(f'{multi_clf_path}portscan')
patator = load(f'{multi_clf_path}patator')
web = load(f'{multi_clf_path}web')

clfs = [dos,ddos,portscan,patator,web]

In [5]:
features = {
    'dos_feature':[ 'Bwd Pkts/s', 'Pkt Len Min', 'Bwd Pkt Len Std', 'FIN Flag Cnt',
                    'Fwd IAT Mean', 'Init Fwd Win Byts', 'Fwd Pkt Len Max', 'Fwd PSH Flags',
                    'SYN Flag Cnt', 'Fwd IAT Min', 'Init Bwd Win Byts', 'Flow IAT Min',
                    'Pkt Size Avg', 'Fwd Seg Size Min', 'Fwd Header Len'],
    'ddos_feature':[ "Fwd Pkt Len Max","Subflow Fwd Byts","TotLen Fwd Pkts"],
    'portscan_feature':[ 'Subflow Fwd Byts','TotLen Fwd Pkts','Fwd Pkt Len Max','Pkt Size Avg'],
    'patator_feature':[ "Init Fwd Win Byts", "Fwd Seg Size Min", "Pkt Len Std",
                         "Init Bwd Win Byts","Flow IAT Min","Pkt Size Avg"],
    'web_feature':[ "Init Bwd Win Byts","Fwd Seg Size Min","Init Fwd Win Byts",
                      "Bwd Pkt Len Std","Fwd IAT Min"]
}

In [28]:
test_file = f'{datapath}mine/normal.pcap_Flow.csv'


In [31]:
# first classify with binary class for anomaly detection
test_dataframe = pd.read_csv(test_file)
# print(test_dataframe['label'].value_counts())
test_dataframe.drop_duplicates(subset=['Flow ID'])
test_dataframe.drop(['Flow ID', 'Src IP', 'Src Port', 'Dst IP','Dst Port','Protocol',
                    'Timestamp','Flow Byts/s', 'Flow Pkts/s','Label'],inplace=True,axis=1)
# test_dataframe.drop(['dst_port','protocol','timestamp','flow_byts_s', 'flow_pkts_s','label'],inplace=True,axis=1)
# test_dataframe.columns = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
#        'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
#        'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
#        'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std',
#        'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
#        'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
#        'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std',
#        'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
#        'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len',
#        'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max',
#        'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt',
#        'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt',
#        'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio',
#        'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
#        'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
#        'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
#        'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
#        'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
#        'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
#        'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
#        'Idle Min']
predictions= bclf.predict(bpca.transform(bscaler.transform(test_dataframe.values)))
print(f'Benign/anomaly ->>> {np.unique(predictions,return_counts=True)}')

for clf_feature,clf in zip(features,clfs):
    y_p = clf.predict(test_dataframe[features[clf_feature]].values)
    print(f'{clf_feature.replace("_feature","")} ->>> {(np.unique(y_p,return_counts=True))}')

Benign/anomaly ->>> (array([1]), array([22]))
dos ->>> (array([1]), array([22]))
ddos ->>> (array([1]), array([22]))
portscan ->>> (array([1]), array([22]))
patator ->>> (array([1]), array([22]))
web ->>> (array([1]), array([22]))
